In [72]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/hyundai/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/hyundai/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/hyundai/x_test.csv")

train = pd.merge(x_train, y_train, on='ID')
target = x_test.copy()

# print(train.shape)
# print()
# print(train.head())
# print()
# print(train.info())
# print()
# print(train.describe())
# print()
# print(train.isnull().sum())

# print(target.shape)
# print()
# print(target.head())
# print()
# print(target.info())
# print()
# print(target.describe())
# print()
# print(target.isnull().sum())

x = train.drop(['ID', 'price'], axis=1)
y = train['price']
target_x = target.drop('ID', axis=1)

# print(target.head(1))
# print(target_x.head(1))

# obj_col = train.select_dtypes(include='object').columns
# for col in obj_col:
#   print(col)
#   print(train[col].unique())
#   print(train[col].nunique())
#   print(target[col].unique())
#   print(target[col].nunique())

df = pd.concat([x,target_x])
# print(df.shape)

from sklearn.preprocessing import StandardScaler, MinMaxScaler
# num_col = df.select_dtypes(exclude='object').columns
# ss = StandardScaler()
# df[num_col] = ss.fit_transform(df[num_col])

num_col = df.select_dtypes(exclude='object').columns
mms = MinMaxScaler()
df[num_col] = mms.fit_transform(df[num_col])

from sklearn.preprocessing import LabelEncoder
obj_col = df.select_dtypes(include='object').columns
obj_col = obj_col.drop('model')

obj = ['model','year']

for col in obj:
  le = LabelEncoder()
  df[col] =le.fit_transform(df[col])

df = pd.get_dummies(df, columns = obj_col, dtype=int)

x_pre = df.iloc[:len(x),:]
target_pre = df.iloc[len(x):,:]

from sklearn.model_selection import train_test_split
rs = 810
x_train, x_test, y_train, y_test = train_test_split(x_pre, y, test_size=0.2, random_state=rs)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=rs)

from sklearn.model_selection import GridSearchCV
param = {'n_estimators':[5,10,15],
         'max_depth':[5,10,15]}
gscv = GridSearchCV(estimator=rf , param_grid=param, cv=5)
gscv.fit(x_train, y_train)
print(gscv.best_params_)
model = gscv.best_estimator_

rf.fit(x_train, y_train)
pred_val = rf.predict(x_test)
pred_val_gs = model.predict(x_test)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
print(mean_squared_error(y_test, pred_val))
print(mean_squared_error(y_test, pred_val, squared=False))
print((mean_squared_error(y_test, pred_val))**0.5)
print(mean_squared_error(y_test, pred_val_gs))

pred = model.predict(target_pre)
result = pd.DataFrame({'ID':target['ID'], 'pred':pred})
result.to_csv('result.csv', index=False)
print(result)

{'max_depth': 10, 'n_estimators': 15}
1485097.9010167462
1218.645929307092
1218.645929307092
1424656.8649283994
       ID          pred
0      21   8444.913612
1      28   4437.434805
2      33  35453.666667
3      38  13809.671106
4      45   8104.721122
..    ...           ...
967  4833   2192.966905
968  4840  18911.910210
969  4842   5076.622222
970  4843   5978.266667
971  4848   7695.491963

[972 rows x 2 columns]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [77]:
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/scipy/rel2.csv')
df.head()

from scipy.stats import shapiro
from scipy.stats import ttest_rel

print(shapiro(df['before']))
print(shapiro(df['after']))
#둘다 정규성 만족

print(ttest_rel(df['before'], df['after']))
#귀무가설 기각 투약전후 변화가 있다

ShapiroResult(statistic=0.9907895174016046, pvalue=0.6065791087935604)
ShapiroResult(statistic=0.9916963493176595, pvalue=0.692386232128563)
TtestResult(statistic=-2.5535473487670677, pvalue=0.011926744724546513, df=119)


In [5]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/scipy/rel1.csv')
df.head()

from scipy.stats import shapiro

print(shapiro(df['before']))
print(shapiro(df['after']))
#정규성 불만족 비모수검정

from scipy.stats import wilcoxon
print(wilcoxon(df['before'], df['after']))
#귀무가설 채택 시험전후 차이가 없음

ShapiroResult(statistic=0.9173727585075375, pvalue=0.0018973571840317808)
ShapiroResult(statistic=0.9448970350592873, pvalue=0.021140909861349116)
WilcoxonResult(statistic=437.0, pvalue=0.12098409484052809)


In [16]:
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/scipy/rel4.csv')
df.head()

print(shapiro(df[df['group']=='a'].rpm))
print(shapiro(df[df['group']=='b'].rpm))
#모두 전규성을 가짐

from scipy.stats import ttest_ind
print(ttest_ind((df[df['group']=='a'].rpm), df[df['group']=='b'].rpm, alternative='greater'))
#귀무가설 기각안함 a가 b보다 크다

ShapiroResult(statistic=0.9907215247460179, pvalue=0.888420220853863)
ShapiroResult(statistic=0.9846746597671578, pvalue=0.5504989397841747)
TtestResult(statistic=-1.8174205730583128, pvalue=0.9643389278690883, df=138.0)


In [28]:
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/scipy/dice.csv')
df.head()

from scipy.stats import chisquare

print(chisquare(df.counts, (df.counts.sum()/6).astype('int')))
#귀무가설 기각 주사위 발생확률은 동일하다

Power_divergenceResult(statistic=2.333333333333333, pvalue=0.8013589222076911)


In [82]:
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/scipy/hands2.csv')
df.head()
target = df.hands.value_counts().to_frame()
print(target)
target['expected']= [int(target.sum()*0.8), int(target.sum()*0.2)]

from scipy.stats import chisquare
print(target)
print(chisquare(target['count'], target['expected']))
#귀무가설 기각 비율이 주어진 값 아님

       count
hands       
right    420
left      80
       count  expected
hands                 
right    420       400
left      80       100
Power_divergenceResult(statistic=5.0, pvalue=0.025347318677468325)


<ipython-input-82-43bab7f129ba>:5: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  target['expected']= [int(target.sum()*0.8), int(target.sum()*0.2)]


In [88]:
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/body/body.csv')
df.head()

from scipy. stats import chi2_contingency
result = chi2_contingency(pd.crosstab(df['측정회원성별'], df['등급']))
print(result)
#귀무가설 기각 성별이랑 등급은 독립이 안미

Chi2ContingencyResult(statistic=120.06233631119409, pvalue=7.481892813401677e-26, dof=3, expected_freq=array([[1220.25, 1220.25, 1220.25, 1220.25],
       [2128.75, 2128.75, 2128.75, 2128.75]]))


In [97]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/scipy/fe2.csv',index_col=0)
df
#관측값 적으니깐 fisher_exact
from scipy.stats import fisher_exact
cdf = df.iloc[:-1,:-1]
print(fisher_exact(cdf))
#귀무가설 기각 참석과 성별은 관련이 있음

SignificanceResult(statistic=18.0, pvalue=0.03571428571428571)


In [101]:
import pandas as pd
df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/consum/Tetuan%20City%20power%20consumption.csv')
df.head()

,DateTime,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone 1 Power Consumption,Zone 2 Power Consumption,Zone 3 Power Consumption
0,1/1/2017 0:00,6.559,73.8,0.083,0.051,0.119,34055.69620,16128.87538,20240.96386
1,1/1/2017 0:10,6.414,74.5,0.083,0.070,0.085,29814.68354,19375.07599,20131.08434
2,1/1/2017 0:20,6.313,74.5,0.080,0.062,0.100,29128.10127,19006.68693,19668.43373
3,1/1/2017 0:30,6.121,75.0,0.083,0.091,0.096,28228.86076,18361.09422,18899.27711
4,1/1/2017 0:40,5.921,75.7,0.081,0.048,0.085,27335.69620,17872.34043,18442.40964


In [105]:
df['DateTime'] = pd.to_datetime(df['DateTime'])
ans = df['DateTime'].dt.month.value_counts().sort_index()
print(ans)

DateTime
1     4464
2     4032
3     4464
4     4320
5     4464
6     4320
7     4464
8     4464
9     4320
10    4464
11    4320
12    4320
Name: count, dtype: int64


In [133]:
df['DateTime'] = pd.to_datetime(df['DateTime'])
target = df[df['DateTime'].dt.month == 3]
ans = target.groupby(target['DateTime'].dt.hour)['Temperature'].mean().sort_values().iloc[0]
print(ans)

11.506612903225806


In [137]:
print(df.columns)

Index(['DateTime', 'Temperature', 'Humidity', 'Wind Speed',
       'general diffuse flows', 'diffuse flows', 'Zone 1 Power Consumption',
       'Zone 2  Power Consumption', 'Zone 3  Power Consumption'],
      dtype='object')


In [139]:
target = df[(df['Zone 1 Power Consumption']) >= (df['Zone 2  Power Consumption'])]
ans = df['Humidity'].mean()
print(ans)

68.25951846764346


In [143]:
target = df[['Zone 1 Power Consumption','Zone 2  Power Consumption', 'Zone 3  Power Consumption']]
ans = target.corr()
print(ans)

AttributeError: 'DataFrame' object has no attribute 'to_frame'

In [148]:
def tt(x):
  if x < 10:
    return 'A'
  elif  10 <= x <20:
    return 'C'
  elif 20 <= x < 30:
    return 'B'
  else:
    return 'D'
df['tt'] = df['Temperature'].apply(tt)
ans = df['tt'].value_counts()
print(ans)

tt
C    26993
B    21105
A     2874
D     1444
Name: count, dtype: int64


In [157]:
df['DateTime'] = pd.to_datetime(df['DateTime'])
target = df[df['DateTime'].dt.month == 6]
ans = target.groupby(target['DateTime'].dt.hour)['Temperature'].std()
print(ans)
ans2 = target[target['DateTime'].dt.hour == 12]
print(ans2.Temperature.std())

DateTime
0     1.118732
1     1.192651
2     1.344856
3     1.443163
4     1.480980
5     1.511074
6     1.365034
7     1.133732
8     1.044652
9     1.233250
10    1.609787
11    1.924741
12    2.049942
13    2.167897
14    2.218134
15    2.268792
16    2.297795
17    2.393686
18    2.184359
19    1.925984
20    1.621100
21    1.415186
22    1.256020
23    1.104244
Name: Temperature, dtype: float64
2.049941782795103


In [159]:
df['DateTime'] = pd.to_datetime(df['DateTime'])
target = df[df['DateTime'].dt.month == 6]
ans2 = target[target['DateTime'].dt.hour == 12]
print(ans2.Temperature.var())

4.202261312849164


In [166]:
target = df[df['Temperature'] >= df['Temperature'].mean()].sort_values('Temperature')
print(target['Humidity'].iloc[3])

87.9


In [167]:
target = df[df['Temperature'] >= df['Temperature'].median()].sort_values('Temperature')
print(target['Humidity'].iloc[3])

80.3


In [168]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/pok/Pokemon.csv')
df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [170]:
target1 = df[df['Legendary'] == True]
target2  =df[df['Legendary'] == False]

ans = target1['HP'].mean() - target2['HP'].mean()
print(ans)

25.55614861329147


In [176]:
ans = df['Type 2'].value_counts().sort_values(ascending=False).index[0]
print(ans)

Flying


In [189]:
target = df['Type 1'].value_counts().sort_values(ascending=False).index[0]
print(target)
ans = df[df['Type 1'] == 'Water']
result = ans['Attack'].mean() / ans['Defense'].mean()
print(result)

Water
1.0165238678090576


In [199]:
target = df[df['Legendary']==True].value_counts('Generation').sort_values(ascending=False).index[0]
print(target)


3


In [238]:
target = df[['HP','Attack','Defense','Sp. Atk', 'Sp. Def', 'Speed']].corr().unstack().sort_values(ascending=False).reset_index().rename(columns={0:'corr'})
print(target[target['corr'] != 1].iloc[0])

level_0     Defense
level_1     Sp. Def
corr       0.510747
Name: 6, dtype: object


In [256]:
target = df.sort_values(['Generation','Attack'], ascending=False).groupby(['Generation'])['Attack'].head(5).mean()
print(target)

157.23333333333332


In [266]:
ans = df[['Type 1', 'Type 2']].value_counts().head(1)
print(ans)

Type 1  Type 2
Normal  Flying    24
Name: count, dtype: int64


In [271]:
target = df[['Type 1', 'Type 2']].value_counts()
print(len(target[target ==1]))

39


In [292]:
target = df[['Type 1','Type 2']].value_counts()
target2 =target[target==1]

lst = []
for value in target2.reset_index().values:
    t1 = value[0]
    t2 = value[1]

    sp = df[(df['Type 1']==t1) & (df['Type 2']==t2)]
    lst.append(sp)

result = pd.concat(lst).reset_index(drop=True).Generation.value_counts().sort_index()
print(result)

Generation
1     1
2     4
3     5
4    13
5     7
6     9
Name: count, dtype: int64


In [343]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/body/body.csv')
df.head()

,측정나이,측정회원성별,신장 : cm,체중 : kg,체지방율 : %,이완기혈압(최저) : mmHg,수축기혈압(최고) : mmHg,악력D : kg,앉아윗몸앞으로굽히기 : cm,교차윗몸일으키기 : 회,제자리 멀리뛰기 : cm,등급
0,59.0,M,175.0,70.6,19.2,91.0,150.0,40.6,12.2,30.0,179.0,C
1,40.0,F,161.4,45.8,21.7,62.0,119.0,22.1,16.0,32.0,165.0,B
2,27.0,M,176.6,84.8,19.2,87.0,153.0,45.9,13.2,61.0,216.0,B
3,38.0,M,167.6,76.7,24.7,63.0,132.0,43.6,16.0,45.0,231.0,A
4,21.0,M,165.2,66.2,21.5,83.0,106.0,33.5,10.6,46.0,198.0,C


In [344]:
target1 = df['수축기혈압(최고) : mmHg']
target2= df['이완기혈압(최저) : mmHg']
print((target1- target2).mean())

52.19539414750672


In [345]:
target = df[(df['측정나이'] >= 50) & (df['측정나이'] <= 59)]
ans = target['신장 : cm'].mean()
print(ans)

164.07490107405314


In [346]:
df['연령대'] = df['측정나이'] // 10 * 10
ans = df['연령대'].value_counts()
print(ans)

연령대
20.0    5831
30.0    2660
40.0    1801
50.0    1769
60.0    1335
Name: count, dtype: int64


In [347]:
target = df.groupby(['연령대','등급'], as_index=False).size()
print(target)

     연령대 등급  size
0   20.0  A  1585
1   20.0  B  1443
2   20.0  C  1455
3   20.0  D  1348
4   30.0  A   743
5   30.0  B   697
6   30.0  C   626
7   30.0  D   594
8   40.0  A   386
9   40.0  B   428
10  40.0  C   455
11  40.0  D   532
12  50.0  A   321
13  50.0  B   410
14  50.0  C   474
15  50.0  D   564
16  60.0  A   314
17  60.0  B   371
18  60.0  C   339
19  60.0  D   311


In [348]:
target = df[df['측정회원성별']=='M']
ans = target[target['등급'] == 'D']['체지방율 : %'].mean() - target[target['등급'] == 'A']['체지방율 : %'].mean()
print(ans)

7.932086486137521


In [349]:
target = df[df['측정회원성별']=='F']
ans = target[target['등급'] == 'D']['체중 : kg'].mean() - target[target['등급'] == 'A']['체중 : kg'].mean()
print(ans)

5.176211590296504


In [353]:
df['bmi'] = df['체중 : kg'] / ((df['신장 : cm']/100)**2)
ans = df[df['측정회원성별']=='M'].bmi.mean()
print(ans)

24.46134409819308


In [354]:

target = df[df['bmi'] < df['체지방율 : %']]['체중 : kg'].mean()
print(target)

61.7408806392542


In [369]:
ans1 = df[df['측정회원성별'] =='M']['악력D : kg'].mean()
ans2 = df[df['측정회원성별'] =='F']['악력D : kg'].mean()
print(ans1 - ans2)

17.55954185047464


In [371]:
ans1 = df[df['측정회원성별'] =='M']['교차윗몸일으키기 : 회'].mean()
ans2 = df[df['측정회원성별'] =='F']['교차윗몸일으키기 : 회'].mean()
print(ans1 - ans2)

14.243156833157627
